# Exploring multi-node cuML performance with Dask on Google Cloud

In this notebook we are going to launch a RAPIDS cluster on Google Cloud with Kubernetes and Dask and then do some performance comparisons.

First we will load some Parquet and CSV data and explore the perforance differences between those file formats.

Then we will run through various cuML algorythms and explore their performance.

## Launch a GPU Kubernetes cluster 

To get started we need to launch a Kubernetes cluster with GPUs on Google Kubernetes Engine.

### Quickstart

```console
$ gcloud container clusters create rapids \
  --accelerator type=nvidia-tesla-a100,count=2 --machine-type a2-highgpu-2g \
  --zone us-central1-c --release-channel stable
```

Then we need to install the NVIDIA drivers.

```console
$ kubectl apply -f https://raw.githubusercontent.com/GoogleCloudPlatform/container-engine-accelerators/master/nvidia-driver-installer/cos/daemonset-preloaded-latest.yaml
daemonset.apps/nvidia-driver-installer created
```

```{docref} /cloud/gcp/gke
For more detailed information on launching GPU powered Kubernetes clusters on Google Cloud see the documentation.
```

Lastly we need to install the Dask Kubernetes Operator.

```console
$ helm install --repo https://helm.dask.org --create-namespace -n dask-operator --generate-name dask-kubernetes-operator
```

```{docref} /tools/kubernetes/dask-operator
For more detailed information on using the Dask Kubernetes Operator see the documentation.
```

### Install dependencies

```{docref} /platforms/kubernetes

You may want to start a Jupyter session in the Kubernetes cluster and run this notebook from there for best performance. Check out the [RAPIDS Interactive Notebook on Kubernetes docs](/platforms/kubernetes) for more information and be sure to follow the "Optional extended notebook configuration".
```

To launch our Dask cluster on Kubernetes we will need `dask-kubernetes` and to access data stored on Google Cloud we will need `gcsfs`.

In [ ]:
!pip install dask-kubernetes gcsfs

Import other dependencies.

In [1]:
import certifi
import cudf
import cuml
import cupy as cp
import numpy as np
import os
import pandas as pd
import random
import seaborn as sns
import time
import yaml

from functools import partial
from math import cos, sin, asin, sqrt, pi
from tqdm import tqdm
from typing import Optional

import dask
import dask.array as da
import dask_cudf

from dask.distributed import Client, wait, progress


class SimpleTimer:
    def __init__(self):
        self.start = None
        self.end = None
        self.elapsed = None

    def __enter__(self):
        self.start = time.perf_counter_ns()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time.perf_counter_ns()
        self.elapsed = self.end - self.start

In [2]:
# Choose the same RAPIDS image you used for launching the notebook session
rapids_image = "{{rapids_container}}"
# Use the number of worker nodes in your Kubernetes cluster
n_workers = 8

### Launch a Dask cluster

Now we can launch a Dask cluster. 

```{note}
To access our remote data we also need to ensure our Dask workers get `gcsfs` installed at runtime.
```

In [ ]:
from dask_kubernetes.operator import KubeCluster

cluster = KubeCluster(
    name="rapids-dask",
    image=rapids_image,
    worker_command="dask-cuda-worker",
    n_workers=n_workers,
    resources={"limits": {"nvidia.com/gpu": "1"}},
    env={
        "DISABLE_JUPYTER": "true",
        "EXTRA_PIP_PACKAGES": "gcsfs",
    },
)
client = Client(cluster)
cluster

In [5]:
client

<Client: 'tcp://10.60.3.8:8786' processes=8 threads=8, memory=1.31 TiB>

## Helper Tools

Define some helper functions for use when exploring `cuML`.

In [6]:
def scale_workers(client, n_workers, timeout=300):
    client.cluster.scale(n_workers)

    m = len(client.has_what().keys())
    start = end = time.perf_counter_ns()
    while (m != n_workers) and (((end - start) / 1e9) < timeout):
        time.sleep(5)
        m = len(client.has_what().keys())

        end = time.perf_counter_ns()

    if ((end - start) / 1e9) >= timeout:
        raise RuntimeError(
            f"Failed to rescale cluster in {timeout} sec."
            "Try increasing timeout for very large containers, and verify available compute resources."
        )


def construct_worker_pool(client, n_workers, auto_scale=False, timeout=300):
    workers = [w for w in client.has_what().keys()]
    if len(workers) < n_workers:
        if auto_scale:
            scale_workers(client=client, n_workers=n_workers, timeout=timeout)
            workers = [w for w in client.has_what().keys()]
        else:
            print(
                "Attempt to construct worker pool larger than available worker set, and auto_scale is False."
                " Returning entire pool."
            )
    else:
        workers = random.sample(population=workers, k=n_workers)

    return workers


def estimate_df_rows(client, files, storage_opts={}, testpct=0.01):
    workers = client.has_what().keys()

    est_size = 0
    for file in files:
        if file.endswith(".csv"):
            df = dask_cudf.read_csv(
                file, npartitions=len(workers), storage_options=storage_opts
            )
        elif file.endswith(".parquet"):
            df = dask_cudf.read_parquet(
                file, npartitions=len(workers), storage_options=storage_opts
            )

        # Select only the index column from our subsample
        est_size += (df.sample(frac=testpct).iloc[:, 0].shape[0] / testpct).compute()
        del df

    return est_size

### Taxi Data Setup

In [7]:
def clean(df_part, remap, must_haves):
    """
    This function performs the various clean up tasks for the data
    and returns the cleaned dataframe.
    """
    tmp = {col: col.strip().lower() for col in list(df_part.columns)}
    df_part = df_part.rename(columns=tmp)

    # rename using the supplied mapping
    df_part = df_part.rename(columns=remap)

    # iterate through columns in this df partition
    for col in df_part.columns:
        # drop anything not in our expected list
        if col not in must_haves:
            df_part = df_part.drop(col, axis=1)
            continue

        # fixes datetime error found by Ty Mckercher and fixed by Paul Mahler
        if df_part[col].dtype == "object" and col in [
            "pickup_datetime",
            "dropoff_datetime",
        ]:
            df_part[col] = df_part[col].astype("datetime64[ms]")
            continue

        # if column was read as a string, recast as float
        if df_part[col].dtype == "object":
            df_part[col] = df_part[col].astype("float32")
        else:
            # downcast from 64bit to 32bit types
            # Tesla T4 are faster on 32bit ops
            if "int" in str(df_part[col].dtype):
                df_part[col] = df_part[col].astype("int32")
            if "float" in str(df_part[col].dtype):
                df_part[col] = df_part[col].astype("float32")
            df_part[col] = df_part[col].fillna(-1)

    return df_part


def coalesce_taxi_data(fraction, random_state):
    base_path = "gcs://anaconda-public-data/nyc-taxi/csv"

    # list of column names that need to be re-mapped
    remap = {}
    remap["tpep_pickup_datetime"] = "pickup_datetime"
    remap["tpep_dropoff_datetime"] = "dropoff_datetime"
    remap["ratecodeid"] = "rate_code"

    # create a list of columns & dtypes the df must have
    must_haves = {
        "pickup_datetime": "datetime64[ms]",
        "dropoff_datetime": "datetime64[ms]",
        "passenger_count": "int32",
        "trip_distance": "float32",
        "pickup_longitude": "float32",
        "pickup_latitude": "float32",
        "rate_code": "int32",
        "dropoff_longitude": "float32",
        "dropoff_latitude": "float32",
        "fare_amount": "float32",
    }

    # apply a list of filter conditions to throw out records with missing or outlier values
    query_frags = [
        "fare_amount > 0 and fare_amount < 500",
        "passenger_count > 0 and passenger_count < 6",
        "pickup_longitude > -75 and pickup_longitude < -73",
        "dropoff_longitude > -75 and dropoff_longitude < -73",
        "pickup_latitude > 40 and pickup_latitude < 42",
        "dropoff_latitude > 40 and dropoff_latitude < 42",
    ]

    valid_months_2016 = [str(x).rjust(2, "0") for x in range(1, 7)]
    valid_files_2016 = [
        f"{base_path}/2016/yellow_tripdata_2016-{month}.csv"
        for month in valid_months_2016
    ]

    df_2014_fractional = dask_cudf.read_csv(
        f"{base_path}/2014/yellow_*.csv", chunksize=25e6
    ).sample(frac=fraction, random_state=random_state)
    df_2014_fractional = clean(df_2014_fractional, remap, must_haves)

    df_2015_fractional = dask_cudf.read_csv(
        f"{base_path}/2015/yellow_*.csv", chunksize=25e6
    ).sample(frac=fraction, random_state=random_state)
    df_2015_fractional = clean(df_2015_fractional, remap, must_haves)

    df_2016_fractional = dask_cudf.read_csv(valid_files_2016, chunksize=25e6).sample(
        frac=fraction, random_state=random_state
    )
    df_2016_fractional = clean(df_2016_fractional, remap, must_haves)

    df_taxi = dask.dataframe.multi.concat(
        [df_2014_fractional, df_2015_fractional, df_2016_fractional]
    )
    df_taxi = df_taxi.query(" and ".join(query_frags))

    return df_taxi

In [8]:
def taxi_csv_data_loader(
    client, response_dtype=np.float32, fraction=1.0, random_state=0
):
    response_id = "fare_amount"
    workers = client.has_what().keys()
    km_fields = [
        "passenger_count",
        "trip_distance",
        "pickup_longitude",
        "pickup_latitude",
        "rate_code",
        "dropoff_longitude",
        "dropoff_latitude",
        "fare_amount",
    ]

    taxi_df = coalesce_taxi_data(fraction=fraction, random_state=random_state)

    taxi_df = taxi_df[km_fields]
    with dask.annotate(workers=set(workers)):
        taxi_df = client.persist(collections=taxi_df)

    X = taxi_df[taxi_df.columns.difference([response_id])].astype(np.float32)
    y = taxi_df[response_id].astype(response_dtype)

    wait(taxi_df)

    return taxi_df, X, y


def taxi_parquet_data_loader(
    client, response_dtype=np.float32, fraction=1.0, random_state=0
):
    # list of column names that need to be re-mapped
    remap = {}
    remap["tpep_pickup_datetime"] = "pickup_datetime"
    remap["tpep_dropoff_datetime"] = "dropoff_datetime"
    remap["ratecodeid"] = "rate_code"

    # create a list of columns & dtypes the df must have
    must_haves = {
        "pickup_datetime": "datetime64[ms]",
        "dropoff_datetime": "datetime64[ms]",
        "passenger_count": "int32",
        "trip_distance": "float32",
        "pickup_longitude": "float32",
        "pickup_latitude": "float32",
        "rate_code": "int32",
        "dropoff_longitude": "float32",
        "dropoff_latitude": "float32",
        "fare_amount": "float32",
    }

    # apply a list of filter conditions to throw out records with missing or outlier values
    query_frags = [
        "fare_amount > 0 and fare_amount < 500",
        "passenger_count > 0 and passenger_count < 6",
        "pickup_longitude > -75 and pickup_longitude < -73",
        "dropoff_longitude > -75 and dropoff_longitude < -73",
        "pickup_latitude > 40 and pickup_latitude < 42",
        "dropoff_latitude > 40 and dropoff_latitude < 42",
    ]

    workers = client.has_what().keys()
    taxi_parquet_path = "gs://anaconda-public-data/nyc-taxi/nyc.parquet"
    response_id = "fare_amount"
    fields = [
        "passenger_count",
        "trip_distance",
        "pickup_longitude",
        "pickup_latitude",
        "rate_code",
        "dropoff_longitude",
        "dropoff_latitude",
        "fare_amount",
    ]

    taxi_df = dask_cudf.read_parquet(taxi_parquet_path, npartitions=len(workers))
    taxi_df = clean(taxi_df, remap, must_haves)
    taxi_df = taxi_df.query(" and ".join(query_frags))
    taxi_df = taxi_df[fields]

    with dask.annotate(workers=set(workers)):
        taxi_df = client.persist(collections=taxi_df)

    wait(taxi_df)

    X = taxi_df[taxi_df.columns.difference([response_id])].astype(np.float32)
    y = taxi_df[response_id].astype(response_dtype)

    return taxi_df, X, y

### Performance Validation Code

In [9]:
def record_elapsed_timings_to_df(
    df, timings, record_template, type, columns, write_to=None
):
    records = [
        dict(record_template, **{"sample_index": i, "elapsed": elapsed, "type": type})
        for i, elapsed in enumerate(timings)
    ]

    df = cudf.concat([df, cudf.DataFrame(records)], ignore_index=True)

    if write_to:
        df.to_csv(write_to, columns=columns)

    return df


def collect_load_time_samples(
    load_func, count, return_final_sample=True, verbose=False
):
    timings = []
    for m in tqdm(range(count)):
        with SimpleTimer() as timer:
            df, X, y = load_func()
        timings.append(timer.elapsed)

    if return_final_sample:
        return df, X, y, timings

    return None, None, None, timings


def collect_func_time_samples(func, count, verbose=False):
    timings = []
    for k in tqdm(range(count)):
        with SimpleTimer() as timer:
            func()
        timings.append(timer.elapsed)

    return timings


def sweep_fit_func(model, func_id, require_compute, X, y, xy_fit, count):
    _fit_func_attr = getattr(model, func_id)
    if require_compute:
        if xy_fit:
            fit_func = partial(lambda X, y: _fit_func_attr(X, y).compute(), X, y)
        else:
            fit_func = partial(lambda X: _fit_func_attr(X).compute(), X)
    else:
        if xy_fit:
            fit_func = partial(_fit_func_attr, X, y)
        else:
            fit_func = partial(_fit_func_attr, X)

    return collect_func_time_samples(func=fit_func, count=count)


def sweep_predict_func(model, func_id, require_compute, X, count):
    _predict_func_attr = getattr(model, func_id)
    predict_func = partial(lambda X: _predict_func_attr(X).compute(), X)

    return collect_func_time_samples(func=predict_func, count=count)


def performance_sweep(
    client,
    model,
    data_loader,
    hardware_type,
    worker_counts=[1],
    samples=1,
    load_samples=1,
    max_data_frac=1.0,
    predict_frac=0.05,
    scaling_type="weak",
    xy_fit=True,
    fit_requires_compute=False,
    update_workers_in_kwargs=True,
    response_dtype=np.float32,
    out_path="./perf_sweep.csv",
    append_to_existing=False,
    model_name=None,
    fit_func_id="fit",
    predict_func_id="predict",
    scaling_denom=None,
    model_args={},
    model_kwargs={},
):
    """
    Primary performance sweep entrypoint.


    Parameters
    ------------
    client: DASK client associated with the cluster we're interesting in collecting performance data for.

    model: Model object on which to gather performance data. This will be created and destroyed,
        once for each element of 'worker_counts'

    data_loader: arbitrary data loading function that will be called to load the appropriate testing data.
        Function that is responsible for loading and returning the data to be used for a given performance run. Function
        signature must accept (client, fraction, and random_state). Client should be used to distribute data, and loaders
        should utilize fraction and random_state with dask's dataframe.sample method to allow for control of how much data
        is loaded.

        When called, its return value should be of the form: df, X, y, where df is the full dask_cudf dataframe, X is a
        dask_cudf dataframe which contains all explanatory variables that will be passed to the 'fit' function, and y is a
        dask_cudf series or dataframe that contains response variables which should be passed to fit/predict as fit(X, y)

    hardware_type: indicates the core hardware the current sweep is running on. ex. 'T4', 'V100', 'A100'

    worker_counts: List indicating the number of workers that should be swept. Ex [1, 2, 4]
        worker counts must fit within the cluster associated with 'client', if the current DASK worker count is different
        from what is requested on a given sweep, attempt to automatically scale the worker count. NOTE: this does not
        mean we will scale the available cluster nodes, just the number of deployed worker pods.

    samples: number of fit/predict samples to record per worker count

    load_samples: number of times to sample data loads. This effectively times how long 'data_loader' runs.

    max_data_frac: maximum fraction of data to return.
        Strong scaling: each run will utilize max_data_frac data.
        Weak scaling: each run will utilize (current worker count) / (max worker count) * max_data_frac data.

    predict_frac: fraction of training data used to test inference

    scaling_type: values can be 'weak' or 'strong' indicating the type of scaling sweep to perform.

    xy_fit: indicates whether or not the model's 'fit' function is of the form (X, y), when xy_fit is False, we assume that
        fit is of the form (X), as is the case with various unsupervised methods ex. KNN.

    fit_requires_compute: False generally, set this to True if the model's 'fit' function requires a corresponding '.compute()'
        call to execute the required work.

    update_workers_in_kwargs: Some algorithms accept a 'workers' list, much like DASK, and will require their kwargs to have
        workers populated. Setting this flag handles this automatically.

    response_dtype: defaults to np.float32, some algorithms require another dtype, such as int32

    out_path: path where performance data csv should be saved

    append_to_existing: When true, append results to an existing csv, otherwise overwrite.

    model_name: Override what we output as the model name

    fit_func_id: Defaults to 'fit', only set this if the model has a non-standard naming.

    predict_func_id: Defaults to 'predict', only set this if the model has a on-standard predict naming.

    scaling_denom: (weak scaling) defaults to max(workers) if unset. Specifies the maximum worker count that weak scaling
        should scale against. For example, when using 1 worker in a weak scaling sweep, the worker will attempt to
        process a fraction of the total data equal to 1/scaling_denom

    model_args: args that will be passed to the model's constructor

    model_kwargs: keyword args that will be passed to the model's constructor

    Returns
    --------

    """

    cols = [
        "n_workers",
        "sample_index",
        "elapsed",
        "type",
        "algorithm",
        "scaling_type",
        "data_fraction",
        "hardware",
    ]
    perf_df = cudf.DataFrame(columns=cols)
    if append_to_existing:
        try:
            perf_df = cudf.read_csv(out_path)
        except:
            pass

    model_name = model_name if model_name else str(model)
    scaling_denom = scaling_denom if (scaling_denom is not None) else max(worker_counts)
    max_data_frac = min(1.0, max_data_frac)

    start_msg = f"Starting {scaling_type}-scaling performance sweep for:\n"
    start_msg += f" model      : {model_name}\n"
    start_msg += f" data loader: {data_loader}.\n"
    start_msg += f"Configuration\n"
    start_msg += "==========================\n"
    start_msg += f"{'Worker counts':<25} : {worker_counts}\n"
    start_msg += f"{'Fit/Predict samples':<25} : {samples}\n"
    start_msg += f"{'Data load samples':<25} : {load_samples}\n"
    start_msg += f"- {'Max data fraction':<23} : {max_data_frac}\n"
    start_msg += f"{'Model fit':<25} : {'X ~ y' if xy_fit else 'X'}\n"
    start_msg += f"- {'Response DType':<23} : {response_dtype}\n"
    start_msg += f"{'Writing results to':<25} : {out_path}\n"
    start_msg += (
        f"- {'Method':<23} : {'overwrite' if not append_to_existing else 'append'}\n"
    )
    print(start_msg, flush=True)

    for n in worker_counts:
        fraction = (
            (n / scaling_denom) * max_data_frac
            if scaling_type == "weak"
            else max_data_frac
        )
        record_template = {
            "n_workers": n,
            "type": "predict",
            "algorithm": model_name,
            "scaling_type": scaling_type,
            "data_fraction": fraction,
            "hardware": hardware_type,
        }
        scale_workers(client, n)

        print(f"Sampling <{load_samples}> load times with {n} workers.", flush=True)

        load_func = partial(
            data_loader,
            client=client,
            response_dtype=response_dtype,
            fraction=fraction,
            random_state=0,
        )
        df, X, y, load_timings = collect_load_time_samples(
            load_func=load_func, count=load_samples
        )

        perf_df = record_elapsed_timings_to_df(
            df=perf_df,
            timings=load_timings,
            type="load",
            record_template=record_template,
            columns=cols,
            write_to=out_path,
        )

        print(
            f"Finished loading <{load_samples}>, samples, to <{n}> workers with a mean time of {np.mean(load_timings)/1e9:0.4f} sec.",
            flush=True,
        )
        print(
            f"Sweeping {model_name} '{fit_func_id}' with <{n}> workers. Sampling <{samples}> times.",
            flush=True,
        )

        if update_workers_in_kwargs and "workers" in model_kwargs:
            model_kwargs["workers"] = workers = list(client.has_what().keys())

        m = model(*model_args, **model_kwargs)
        if fit_func_id:
            fit_timings = sweep_fit_func(
                model=m,
                func_id=fit_func_id,
                require_compute=fit_requires_compute,
                X=X,
                y=y,
                xy_fit=xy_fit,
                count=samples,
            )

            perf_df = record_elapsed_timings_to_df(
                df=perf_df,
                timings=fit_timings,
                type="fit",
                record_template=record_template,
                columns=cols,
                write_to=out_path,
            )

            print(
                f"Finished gathering <{samples}>, 'fit' samples using <{n}> workers, with a mean time of {np.mean(fit_timings)/1e9:0.4f} sec.",
                flush=True,
            )
        else:
            print(f"Skipping fit sweep, fit_func_id is None")

        if predict_func_id:
            print(
                f"Sweeping {model_name} '{predict_func_id}' with <{n}> workers. Sampling <{samples}> times.",
                flush=True,
            )
            predict_timings = sweep_predict_func(
                model=m,
                func_id=predict_func_id,
                require_compute=True,
                X=X,
                count=samples,
            )

            perf_df = record_elapsed_timings_to_df(
                df=perf_df,
                timings=predict_timings,
                type="predict",
                record_template=record_template,
                columns=cols,
                write_to=out_path,
            )

            print(
                f"Finished gathering <{samples}>, 'predict' samples using <{n}> workers, with a mean time of {np.mean(predict_timings)/1e9:0.4f} sec.",
                flush=True,
            )
        else:
            print(f"Skipping inference sweep. predict_func_id is None")

### Vis and Analysis

In [10]:
def simple_ci(df, fields, groupby):
    gbdf = df[fields].groupby(groupby).agg(["mean", "std", "count"])

    ci = 1.96 + gbdf["elapsed"]["std"] / np.sqrt(gbdf["elapsed"]["count"])

    ci_df = ci.reset_index()
    ci_df["ci.low"] = gbdf["elapsed"].reset_index()["mean"] - ci_df[0]
    ci_df["ci.high"] = gbdf["elapsed"].reset_index()["mean"] + ci_df[0]

    return ci_df


def visualize_csv_data(csv_path):
    df = cudf.read_csv(csv_path)

    fields = ["elapsed", "elapsed_sec", "type", "n_workers", "hardware", "scaling_type"]
    groupby = ["n_workers", "type", "hardware", "scaling_type"]
    df["elapsed_sec"] = df["elapsed"] / 1e9

    ci_df = simple_ci(df, fields, groupby=groupby)

    # Rescale to seconds
    ci_df[["ci.low", "ci.high"]] = ci_df[["ci.low", "ci.high"]] / 1e9

    # Print confidence intervals
    print(
        ci_df[["hardware", "n_workers", "type", "ci.low", "ci.high"]][
            ci_df["type"] != "load"
        ]
    )

    sns.set_theme(style="whitegrid")
    sns.set(rc={"figure.figsize": (20, 10)}, font_scale=2)

    # Boxplots for elapsed time at each worker count.
    plot_df = df[fields][df[fields].type != "load"].to_pandas()
    ax = sns.catplot(
        data=plot_df,
        x="n_workers",
        y="elapsed_sec",
        col="type",
        row="scaling_type",
        hue="hardware",
        kind="box",
        height=8,
        order=None,
    )

### Taxi Data Configuration (Medium)
We can use the parquet data from the anaconda public repo here. Which will illustrate how much faster it is to read parquet, and gives us around 150 million rows of data to work with.

In [11]:
# Uncomment to test with Taxi Dataset
preload_data = False
append_to_existing = True
samples = 5
load_samples = 1
worker_counts = [8]
scaling_denom = 8
hardware_type = "A100"
max_data_frac = 1.0
scale_type = "weak"  # weak | strong
out_prefix = "taxi_medium"

if not preload_data:
    data_loader = taxi_parquet_data_loader
else:
    data = taxi_parquet_data_loader(client, fraction=max_data_frac)
    data_loader = lambda client, response_dtype, fraction, random_state: data

if not hardware_type:
    raise RuntimeError(
        "Please specify the hardware type for this run! ex. (T4, V100, A100)"
    )

sweep_kwargs = {
    "append_to_existing": append_to_existing,
    "samples": samples,
    "load_samples": load_samples,
    "worker_counts": worker_counts,
    "scaling_denom": scaling_denom,
    "hardware_type": hardware_type,
    "data_loader": data_loader,
    "max_data_frac": max_data_frac,
    "scaling_type": scale_type,
}

In [12]:
taxi_parquet_path = ["gs://anaconda-public-data/nyc-taxi/nyc.parquet"]
estimated_rows = estimate_df_rows(client, files=taxi_parquet_path, testpct=0.0001)
print(estimated_rows)

146170000.0


### Taxi Data Configuration (Large)
The largest dataset we'll work with, contains up to 450 million rows of taxi data, stored as CSV files.

In [13]:
# Uncomment to sweep with the large Taxi Dataset
preload_data = True
append_to_existing = True
samples = 5
load_samples = 1
worker_counts = [8]
scaling_denom = 8
hardware_type = "A100"
data_loader = taxi_csv_data_loader
max_data_frac = 1.0
scale_type = "weak"
out_prefix = "taxi_large"


if not preload_data:
    data_loader = taxi_csv_data_loader
else:
    data = taxi_csv_data_loader(client, fraction=max_data_frac)
    data_loader = lambda client, response_dtype, fraction, random_state: data

if not hardware_type:
    raise RuntimeError(
        "Please specify the hardware type for this run! ex. (T4, V100, A100)"
    )

sweep_kwargs = {
    "append_to_existing": append_to_existing,
    "samples": samples,
    "load_samples": load_samples,
    "worker_counts": worker_counts,
    "scaling_denom": scaling_denom,
    "hardware_type": hardware_type,
    "data_loader": data_loader,
    "max_data_frac": max_data_frac,
    "scaling_type": scale_type,
}

## ETL Exploration CSV vs Parquet

Let's load in some data from Google Cloud Storage in both CSV and Parquet and compare the performance differences.

In [14]:
remap = {}
remap["tpep_pickup_datetime"] = "pickup_datetime"
remap["tpep_dropoff_datetime"] = "dropoff_datetime"
remap["ratecodeid"] = "rate_code"

# create a list of columns & dtypes the df must have
must_haves = {
    "pickup_datetime": "datetime64[ms]",
    "dropoff_datetime": "datetime64[ms]",
    "passenger_count": "int32",
    "trip_distance": "float32",
    "pickup_longitude": "float32",
    "pickup_latitude": "float32",
    "rate_code": "int32",
    "dropoff_longitude": "float32",
    "dropoff_latitude": "float32",
    "fare_amount": "float32",
}

# apply a list of filter conditions to throw out records with missing or outlier values
query_frags = [
    "fare_amount > 0 and fare_amount < 500",
    "passenger_count > 0 and passenger_count < 6",
    "pickup_longitude > -75 and pickup_longitude < -73",
    "dropoff_longitude > -75 and dropoff_longitude < -73",
    "pickup_latitude > 40 and pickup_latitude < 42",
    "dropoff_latitude > 40 and dropoff_latitude < 42",
]

workers = client.has_what().keys()

In [15]:
base_path = "gcs://anaconda-public-data/nyc-taxi/csv"

with SimpleTimer() as timer_csv:
    df_csv_2014 = dask_cudf.read_csv(
        f"{base_path}/2014/yellow_*.csv",
        chunksize=25e6,
        dtype={" tolls_amount": "float64"},
    )
    df_csv_2014 = clean(df_csv_2014, remap, must_haves)
    df_csv_2014 = df_csv_2014.query(" and ".join(query_frags))

    with dask.annotate(workers=set(workers)):
        df_csv_2014 = client.persist(collections=df_csv_2014)

    wait(df_csv_2014)

print(df_csv_2014.columns)
rows_csv = df_csv_2014.iloc[:, 0].shape[0].compute()
print(
    f"CSV load took {timer_csv.elapsed/1e9} sec. For {rows_csv} rows of data => {rows_csv/(timer_csv.elapsed/1e9)} rows/sec"
)

Index(['pickup_datetime', 'dropoff_datetime', 'passenger_count',
       'trip_distance', 'pickup_longitude', 'pickup_latitude', 'rate_code',
       'dropoff_longitude', 'dropoff_latitude', 'fare_amount'],
      dtype='object')
CSV load took 105.232076116 sec. For 155485869 rows of data => 1477552.0424837382 rows/sec


In [16]:
client.cancel(df_csv_2014)

In [17]:
with SimpleTimer() as timer_parquet:
    df_parquet = dask_cudf.read_parquet(
        f"gs://anaconda-public-data/nyc-taxi/nyc.parquet", chunksize=25e6
    )
    df_parquet = clean(df_parquet, remap, must_haves)
    df_parquet = df_parquet.query(" and ".join(query_frags))

    with dask.annotate(workers=set(workers)):
        df_parquet = client.persist(collections=df_parquet)

    wait(df_parquet)

print(df_parquet.columns)
rows_parquet = df_parquet.iloc[:, 0].shape[0].compute()
print(
    f"Parquet load took {timer_parquet.elapsed/1e9} sec. For {rows_parquet} rows of data => {rows_parquet/(timer_parquet.elapsed/1e9)} rows/sec"
)

/opt/conda/envs/rapids/lib/python3.9/site-packages/dask/dataframe/io/parquet/core.py:371: FutureWarning: The `chunksize` argument will be deprecated in the future. Please use `split_row_groups` to specify how many row-groups should be mapped to each output partition.

If you strongly oppose the deprecation of `chunksize`, please comment at https://github.com/dask/dask/issues/9043
  warnings.warn(


Index(['dropoff_datetime', 'passenger_count', 'trip_distance',
       'pickup_longitude', 'pickup_latitude', 'rate_code', 'dropoff_longitude',
       'dropoff_latitude', 'fare_amount'],
      dtype='object')
Parquet load took 33.298469504 sec. For 138367618 rows of data => 4155374.7082393225 rows/sec


In [18]:
client.cancel(df_parquet)

In [19]:
speedup = (rows_parquet / (timer_parquet.elapsed / 1e9)) / (
    rows_csv / (timer_csv.elapsed / 1e9)
)
print(speedup)

2.812337290843721


## cuML Algorithms -- Performance Sweeps

Now let's explore the performance of running various algorithms from `cuML`.

### RandomForestRegressor

In [ ]:
from cuml.dask.ensemble import RandomForestRegressor

rf_kwargs = {"workers": client.has_what().keys(), "n_estimators": 10, "max_depth": 12}
rf_csv_path = f"./{out_prefix}_random_forest_regression.csv"

performance_sweep(
    client=client,
    model=RandomForestRegressor,
    **sweep_kwargs,
    out_path=rf_csv_path,
    response_dtype=np.int32,
    model_kwargs=rf_kwargs,
)

In [ ]:
rf_csv_path = f"./{out_prefix}_random_forest_regression.csv"
visualize_csv_data(rf_csv_path)

### KMeans

In [ ]:
from cuml.dask.cluster import KMeans

kmeans_kwargs = {
    "client": client,
    "n_clusters": 12,
    "max_iter": 371,
    "tol": 1e-5,
    "oversampling_factor": 3,
    "max_samples_per_batch": 32768 / 2,
    "verbose": False,
    "init": "random",
}
kmeans_csv_path = f"./{out_prefix}_kmeans.csv"

performance_sweep(
    client=client,
    model=KMeans,
    **sweep_kwargs,
    out_path=kmeans_csv_path,
    xy_fit=False,
    model_kwargs=kmeans_kwargs,
)

In [ ]:
visualize_csv_data(kmeans_csv_path)

### Nearest Neighbors

In [ ]:
from cuml.dask.neighbors import NearestNeighbors

nn_kwargs = {}
nn_csv_path = f"./{out_prefix}_nn.csv"

performance_sweep(
    client=client,
    model=NearestNeighbors,
    **sweep_kwargs,
    out_path=nn_csv_path,
    xy_fit=False,
    predict_func_id="get_neighbors",
    model_kwargs=nn_kwargs,
)

In [ ]:
nn_csv_path = f"./{out_prefix}_nn.csv"
visualize_csv_data(nn_csv_path)

### PCA

In [ ]:
from cuml.dask.decomposition import PCA

pca_kwargs = {"client": client, "n_components": 5, "whiten": False}
pca_csv_path = f"./{out_prefix}_pca.csv"

performance_sweep(
    client=client,
    model=PCA,
    **sweep_kwargs,
    out_path=pca_csv_path,
    xy_fit=False,
    fit_requires_compute=True,
    fit_func_id="fit_transform",
    predict_func_id=None,  # PCA has no 'predict' method.
    model_kwargs=pca_kwargs,
)

In [ ]:
visualize_csv_data(pca_csv_path)

### TruncatedSVD

In [ ]:
from cuml.dask.decomposition import TruncatedSVD

tsvd_kwargs = {"client": client, "n_components": 5}
tsvd_csv_path = f"./{out_prefix}_tsvd.csv"

performance_sweep(
    client=client,
    model=TruncatedSVD,
    **sweep_kwargs,
    out_path=tsvd_csv_path,
    xy_fit=False,
    fit_requires_compute=True,
    fit_func_id="fit_transform",
    predict_func_id=None,
    model_kwargs=tsvd_kwargs,
)

In [ ]:
visualize_csv_data(tsvd_csv_path)

### Linear Regression

In [ ]:
from cuml.dask.linear_model import LinearRegression

lr_kwargs = {"client": client, "algorithm": "eig"}

lr_csv_path = f"./{out_prefix}_linear_regression.csv"

performance_sweep(
    client=client,
    model=LinearRegression,
    **sweep_kwargs,
    out_path=lr_csv_path,
    model_kwargs=lr_kwargs,
)

In [ ]:
visualize_csv_data(lr_csv_path)

### Ridge Regression

In [ ]:
from cuml.dask.linear_model import Ridge as RidgeRegression

ridge_kwargs = {"client": client, "solver": "eig"}

ridge_csv_path = f"./{out_prefix}_ridge_regression.csv"

performance_sweep(
    client=client,
    model=RidgeRegression,
    **sweep_kwargs,
    out_path=ridge_csv_path,
    model_kwargs=ridge_kwargs,
)

In [ ]:
visualize_csv_data(ridge_csv_path)

### Lasso Regression

In [ ]:
from cuml.dask.linear_model import Lasso as LassoRegression

lasso_kwargs = {"client": client}

lasso_csv_path = f"./{out_prefix}_lasso_regression.csv"

performance_sweep(
    client=client,
    model=LassoRegression,
    **sweep_kwargs,
    out_path=lasso_csv_path,
    model_kwargs=lasso_kwargs,
)

In [ ]:
visualize_csv_data(lasso_csv_path)

### ElasticNet Regression

In [ ]:
from cuml.dask.linear_model import ElasticNet as ElasticNetRegression

elastic_kwargs = {
    "client": client,
}

enr_csv_path = f"./{out_prefix}_elastic_regression.csv"

performance_sweep(
    client=client,
    model=ElasticNetRegression,
    **sweep_kwargs,
    out_path=enr_csv_path,
    model_kwargs=elastic_kwargs,
)

In [ ]:
visualize_csv_data(enr_csv_path)

### Model Parallel Multi-GPU Linear Regression

In [ ]:
from cuml.dask.solvers import CD

cd_kwargs = {}

cd_csv_path = f"./{out_prefix}_mutli_gpu_linear_regression.csv"

performance_sweep(
    client=client,
    model=CD,
    **sweep_kwargs,
    out_path=cd_csv_path,
    model_kwargs=cd_kwargs,
)

In [ ]:
visualize_csv_data(cd_csv_path)

### XGBoost

In [ ]:
import xgboost as xgb

xg_args = [client]
xg_kwargs = {
    "params": {
        "tree_method": "gpu_hist",
    },
    "num_boost_round": 100,
}

xgb_csv_path = f"./{out_prefix}_xgb.csv"


class XGBProxy:
    """
    Create a simple API wrapper around XGBoost so that it supports the fit/predict workflow.

    Parameters
    -------------
    data_loader: data loader object intended to be used by the performance sweep.
    """

    def __init__(self, data_loader):
        self.args = []
        self.kwargs = {}
        self.data_loader = data_loader
        self.trained_model = None

    def loader(self, client, response_dtype, fraction, random_state):
        """
        Wrap the data loader method so that it creates a DMatrix from the returned data.
        """
        df, X, y = self.data_loader(client, response_dtype, fraction, random_state)
        dmatrix = xgb.dask.DaskDMatrix(client, X, y)

        return dmatrix, dmatrix, dmatrix

    def __call__(self, *args, **kwargs):
        """
        Acts as a pseudo init function which initializes our model args.
        """
        self.args = args
        self.kwargs = kwargs

        return self

    def fit(self, X):
        """
        Wrap dask.train, and store the model on our proxy object.
        """
        if self.trained_model:
            del self.trained_model

        self.trained_model = xgb.dask.train(
            *self.args, dtrain=X, evals=[(X, "train")], **self.kwargs
        )
        return self

    def predict(self, X):
        assert self.trained_model

        return xgb.dask.predict(*self.args, self.trained_model, X)


xgb_proxy = XGBProxy(data_loader)
performance_sweep(
    client=client,
    model=xgb_proxy,
    data_loader=xgb_proxy.loader,
    hardware_type=hardware_type,
    worker_counts=worker_counts,
    samples=samples,
    load_samples=load_samples,
    max_data_frac=max_data_frac,
    scaling_type=scale_type,
    out_path=xgb_csv_path,
    append_to_existing=append_to_existing,
    update_workers_in_kwargs=False,
    xy_fit=False,
    scaling_denom=scaling_denom,
    model_args=xg_args,
    model_kwargs=xg_kwargs,
)

In [ ]:
visualize_csv_data(xgb_csv_path)